# Preamble

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import wikitextparser as wtp
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
from datetime import datetime
from ast import literal_eval

# Lists

In [23]:
# API variables
api_url = 'https://yugipedia.com/api.php'
sets_query_url = '?action=ask&query=[[Category:Set%20Card%20Lists]]|limit%3D5000|order%3Dasc&format=json'
lists_query_url = '?action=query&prop=revisions&rvprop=content&format=json&titles='

abbreviations_dict = {'c': 'Common', 'r': 'Rare', 'sr': 'Super Rare', 'ur': 'Ultra Rare', 'utr': 'Ultimate Rare', 'n': 'Normal', 'nr': 'Normal Rare', 'sp': 'Short Print', 'ssp': 'Super Short Print', 'hfr': 'Holofoil Rare', 'scr': 'Secret Rare', 'uscr': 'Ultra Secret Rare', 'scur': 'Secret Ultra Rare', 'pscr': 'Prismatic Secret Rare', 'hgr': 'Holographic Rare', ' gr': 'Ghost Rare', 'pr': 'Parallel Rare', 'npr': 'Normal Parallel Rare', 'pc': 'Parallel Common', 'spr': 'Super Parallel Rare', 'upr': 'Ultra Parallel Rare', 'dnpr': 'Duel Terminal Normal Parallel Rare', 'dpc': 'Duel Terminal Parallel Common', 'drpr': 'Duel Terminal Rare Parallel Rare', 'dspr': 'Duel Terminal Super Parallel Rare', 'dupr': 'Duel Terminal Ultra Parallel Rare', 'DScPR': 'Duel Terminal Secret Parallel Rare', 'gur': 'Gold Rare', 'escr': 'Extra Secret Rare', 'ggr': 'Ghost/Gold Rare', 'shr': 'Shatterfoil Rare', 'cr': 'Collector\'s Rare', 'str': 'Starlight Rare', 'gr': 'Ghost Rare', 'gscr': 'Gold Secret Rare', 'sfr': 'Starfoil Rare'}

In [ ]:
'https://yugipedia.com/api.php?action=askargs&conditions=Legend%20of%20Blue%20Eyes%20White%20Dragon&printouts=German%20release%20date'

In [24]:
df = pd.read_json(f'{api_url}{sets_query_url}')
keys = list(df['query']['results'].keys())
print(len(keys))

3943


In [25]:
all_set_lists = pd.DataFrame(columns = ['Set','Card number','Name','Rarity','Print','Quantity'])

for i in range(40):
    first = i*100
    last = (i+1)*100
    titles = '|'.join(keys[first:last]).replace(' ','%20').encode('utf-8')
    df = pd.read_json(f'{api_url}{lists_query_url}{titles}')
    contents = df['query']['pages'].values()
    
    for content in contents:
        if 'revisions' in  content.keys():
            temp = content['revisions'][0]['*']
            parsed = wtp.parse(temp)
            
            for template in parsed.templates:
                if template.name == 'Set list':
                    title = content['title'].split('Lists:')[1]
                    set_df = pd.DataFrame(columns = ['Set', 'Card number', 'Name', 'Rarity', 'Print', 'Quantity'])


                    region = None
                    rarity = None
                    card_print = None
                    qty = None
                    desc = None
                    opt = None
                    list_df = None
                    
                    for argument in template.arguments:
                        if 'region=' in argument:
                            region = argument.string[argument.string.index('=')+1:]
                        elif 'rarities=' in argument:
                            rarity = tuple(abbreviations_dict.get(i.strip().lower(), i.strip()) for i in argument.string[argument.string.index('=')+1:].split(','))
                        elif 'print=' in argument:
                            card_print = argument.string[argument.string.index('=')+1:]
                        elif 'qty=' in argument:
                            qty = argument.string[argument.string.index('=')+1:]
                        elif 'description=' in argument:
                            desc = argument.string[argument.string.index('=')+1:]
                        elif 'options=' in argument:
                            opt = argument.string[argument.string.index('=')+1:]
                        else:
                            set_list = argument.string[2:-1]
                            lines = set_list.split('\n')

                            list_df = pd.DataFrame([x.split(';') for x in lines])
                            list_df = list_df[~list_df[0].str.contains('!:')]
                            list_df = list_df.applymap(lambda x: x.split('//')[0] if x is not None else x)
                            list_df = list_df.applymap(lambda x: x.strip() if x is not None else x)
                            list_df.replace(r'^\s*$', None, regex = True, inplace = True)

                    if opt != 'noabbr':
                        set_df['Card number'] = list_df[0]
                        set_df['Name'] = list_df[1]
                    else: 
                        set_df['Name'] = list_df[0]

                    if len(list_df.columns)>2: # and rare in str
                        set_df['Rarity'] = list_df[2].apply(lambda x: tuple([abbreviations_dict.get(y.strip().lower(), y.strip()) for y in x.split(',')]) if x is not None else rarity)
                    else:
                        set_df['Rarity'] = [rarity for _ in set_df.index]

                    if len(list_df.columns)>3 :
                        if card_print is not None: # and new/reprint in str
                            set_df['Print'] = list_df[3].apply(lambda x: x if x is not None else card_print)
                            if len(list_df.columns)>4 and qty is not None:
                                set_df['Quantity'] = list_df[4].apply(lambda x: x if x is not None else qty)
                        elif qty is not None:
                            set_df['Quantity'] = list_df[3].apply(lambda x: x if x is not None else qty)
                    
                    set_df['Set'] = title
                    all_set_lists = pd.concat([all_set_lists, set_df], ignore_index=True)

In [26]:
all_set_lists

,Set,Card number,Name,Rarity,Print,Quantity
0,2013 Collectible Tins Wave 2 (TCG-DE),CT10-DE003,"Redox, Dragon Ruler of Boulders","(Secret Rare,)",NaN,NaN
1,2013 Collectible Tins Wave 2 (TCG-DE),CT10-DE012,Diamond Dire Wolf,"(Super Rare,)",NaN,NaN
2,2013 Collectible Tins Wave 2 (TCG-DE),CT10-DE013,Number 88: Gimmick Puppet of Leo,"(Super Rare,)",NaN,NaN
3,2013 Collectible Tins Wave 2 (TCG-DE),CT10-DE014,Spellbook of the Master,"(Super Rare,)",NaN,NaN
4,2013 Collectible Tins Wave 2 (TCG-DE),CT10-DE004,"Tempest, Dragon Ruler of Storms","(Secret Rare,)",NaN,NaN
...,...,...,...,...,...,...
71843,King's Court (TCG-EN),KICO-EN061,The Wicked Avatar,"(Rare,)",Reprint,NaN
71844,King's Court (TCG-EN),KICO-EN062,The Wicked Eraser,"(Rare,)",Reprint,NaN
71845,King's Court (TCG-EN),KICO-EN063,Slifer the Sky Dragon,"(Ultra Rare,)",Reprint,NaN
71846,King's Court (TCG-EN),KICO-EN064,Obelisk the Tormentor,"(Ultra Rare,)",Reprint,NaN


In [ ]:
all_set_lists.groupby('Card number', dropna=False).nunique()

In [ ]:
all_set_lists.groupby('Name', dropna=False).nunique()

In [30]:
all_set_lists.explode('Rarity').groupby('Rarity').nunique()

,Set,Card number,Name,Print,Quantity
Rarity,,,,,
10000 Secret Rare,5,5,1,1,0
20th Secret Rare,11,68,67,2,0
20th Secret rare,1,2,2,0,0
Collector's Rare,2,30,15,2,0
Collectors Rare,1,45,45,0,0
Common,832,35562,6816,2,3
Duel Terminal Normal Parallel Rare,13,554,394,0,0
Duel Terminal Normal Rare Parallel Rare,1,6,6,0,0
Duel Terminal Rare Parallel Rare,11,152,96,0,0


In [ ]:
all_set_lists.groupby('Print', dropna=False).nunique()

In [ ]:
all_set_lists.groupby('Quantity', dropna=False).nunique()

In [ ]:
all_set_lists[all_set_lists['Card number']=='!: header::Kaiba Deck']